In [ ]:
from pydub import AudioSegment
from pydub.playback import play
from pydub import AudioSegment
import numpy as np
import os
from surfboard.sound import Waveform
import json
import pyannote.core.json
from pyannote.core import Segment, Timeline
from surfboard.feature_extraction import extract_features
from surfboard.feature_extraction import extract_features_from_waveform
import pandas as pd  

In [ ]:
os.chdir('/Users/andrei-macpro/Documents/Data/Audio/Embeddings/Play/identification/speech_turns_cg')  #if string starts with slash it is considered absolute
file_names=sorted([i for i in os.listdir(".") if not i.startswith(".")])

In [ ]:
# function that takes as input a list of file-names and outputs a list of pyannote timelines 
path = '/Users/andrei-macpro/Documents/Data/Audio/Embeddings/Play/identification/speech_turns_cg'
json_files = [pos_json for pos_json in sorted(os.listdir(path))]
def get_timelines(json_files):
    list_json=list()
    list_timelines=list()
    for filename in json_files: # loop that imports all json data into separate dictionaries
        with open(filename) as json_file:
            data = json.load(json_file)
            list_json.append(data)
    for file, segments in zip(json_files, list_json): # loop to iterate through the files and create a new timeline for each of them 
        timeline=Timeline()
        list_timelines.append(timeline.from_json(segments))
    return list_timelines

In [ ]:
indexes = [int(file.split('.')[0]) for file in file_names]

In [ ]:
list_timelines=get_timelines(json_files)

In [ ]:
os.chdir('/Users/andrei-macpro/Documents/Data/Audio/overlapping_speech/no_os_play')  #if string starts with slash it is considered absolute
file_names=sorted([i for i in os.listdir(".") if not i.startswith(".")])

In [ ]:
file_names = [i for i in file_names if int(i.split('_')[0]) in indexes]
file_names

In [ ]:
trial = AudioSegment.from_wav('1043_meal.wav')

In [ ]:
sound = Waveform('trial_1043.wav')

In [ ]:
trial2 = trial.export("trial_1047.wav", format="wav")

In [ ]:
list_timelines[0]

In [ ]:
waveforms = []
for index, segments in enumerate(list_timelines[0]):
    seg = (trial[segments.start*1000:segments.end*1000])
    seg.export('seg.wav', format='wav')
    waveforms.append(Waveform('seg.wav'))

In [ ]:
len(waveforms)

In [ ]:
file_names, list_timelines

In [ ]:
len(list_timelines[0])

In [ ]:
trial = AudioSegment.from_wav('1043_meal_child.wav')
trial[list_timelines[0][0].start*1000:list_timelines[0][0].end*1000].export('trial.wav', format="wav")

In [ ]:
sound = Waveform('trial.wav')

In [ ]:
waveforms = []
for file, timeline in zip(file_names, list_timelines):
    trial = AudioSegment.from_wav(file)
    temp_list=[]
    for index, segments in enumerate(timeline):
        
        seg = (trial[segments.start*1000:segments.end*1000])
        seg.export('seg.wav', format='wav')
        try:
            temp_list.append(Waveform('seg.wav'))
        except Exception:
            pass
        print(index,file)
    waveforms.append(temp_list)
        

In [ ]:
len(waveforms[0])

In [ ]:
components = ['mfcc', 'zerocrossing', 'rms', 'f0_contour']
statistics = ['mean', 'first_derivative_mean', 'std']


     
# Extract dataframe...
feature_df = extract_features(
    waveforms=[waveforms[0][0]], components_list=components, statistics_list=statistics
)

In [ ]:
features = []
for participant in waveforms:
    temp_list=[]
    for waveform in participant:
        temp_list.append(extract_features_from_waveform(components, statistics, waveform))
    features.append(temp_list)

In [ ]:
features[0][0].keys()

In [ ]:

pd.Series(features[0][0]).to_frame()

In [ ]:
df = pd.DataFrame(features[0])  

In [ ]:
df

In [ ]:
dfs = []
for participant, index in zip(features, indexes):
    dfs.append(pd.DataFrame(participant))

In [ ]:
for ind in indexes:
    dfs[indexes.index(ind)]['s_id'] = ind

dfs[0]


In [ ]:
for df in range(len(dfs)):
    dfs[df] = dfs[df].set_index('s_id')

dfs[0]

In [ ]:
os.chdir('/Users/andrei-macpro/Documents/Data/Results/Accoustics/cg_play_surfboard')

In [ ]:
for df, ind in zip(dfs, indexes):
    df.to_csv(str(ind)+'.csv')